In [1]:
!pip list | grep torch

In [ ]:
%pip install gradio
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api
%pip install --upgrade --quiet llama-index-llms-nvidia llama-index-embeddings-nvidia llama-index-readers-file

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 13.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 14.2 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install "transformers[torch]" "huggingface_hub[inference]"

In [ ]:
!pip install llama-index

In [ ]:
hf_token = ""

import getpass
import os

# del os.environ['HF_TOKEN']  ## delete key and reset
if os.environ.get("HF_TOKEN", "").startswith("hf_"):
    print("Valid HF_TOKEN already in environment. Delete to reset")
else:
    hf_token = getpass.getpass("HF TOKEN (starts with hf_): ")
    assert hf_token.startswith(
        "hf_"
    ), f"{hf_token[:5]}... is not a valid key"
    os.environ["HF_TOKEN"] = hf_token

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    token=os.environ.get("HF_TOKEN", ""),
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

In [ ]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM

# Optional quantization to 4bit
# import torch
# from transformers import BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

local_llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    model_kwargs={
        "token": hf_token,
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
        # "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer_name="meta-llama/Llama-3.1-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

In [ ]:
response = local_llm.complete("What is Dell Technologies?")

print(response)

In [ ]:
content = ""
for completion in local_llm.stream_complete("What is Dell Technologies?"):
    content += completion.delta
    print(completion.delta, end="")

In [ ]:
# import gradio as gr

# def stream_response_local(message, history):
#     response = local_llm.stream_complete(message)
#     res = ""
#     for token in response:
#         # print(token, end="")
#         res = str(res) + str(token.delta)
#         yield res

# with gr.Blocks() as demo1:
#     gr.Markdown(
#     """
#     <h1 style="text-align: center;">Local HuggingFaceLLM Chatbot 💻📑✨</h3>
#     """)
#     with gr.Row(equal_height=False):
#         with gr.Column():
#             test = gr.ChatInterface(fn=stream_response_local)
            

# # demo.launch(server_name="0.0.0.0", ssl_verify=False)
# demo1.launch()



In [ ]:
import getpass
import os

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith(
        "nvapi-"
    ), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

In [ ]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.llms.nvidia import NVIDIA

# connect to an chat NIM running at localhost:8080, spcecifying a specific model
hosted_nim_llm = NVIDIA(
    model="meta/llama-3.1-8b-instruct"
)

content = ""
for completion in hosted_nim_llm.stream_complete("What is Dell Technologies?"):
    content += completion.delta
    print(completion.delta, end="")

In [ ]:
from llama_index.llms.nvidia import NVIDIA

# connect to an chat NIM running at localhost:8080, spcecifying a specific model
local_nim_llm = NVIDIA(
    base_url="http://localhost:8000/v1", model="meta/llama-3.1-8b-instruct"
)

content = ""
for completion in local_nim_llm.stream_complete("What is Dell Technologies?"):
    content += completion.delta
    print(completion.delta, end="")